In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv2D, Dropout, MaxPool2D, Flatten
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, r2_score
import numpy as np

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('sample_submission.csv', index_col=0)

In [3]:
x1 = train.iloc[:,:1]

x2 = train.iloc[:,1:36]

x3 = train.iloc[:,36:71]

y1 = train.iloc[:,71:72]

y2 = train.iloc[:,72:73]

y3 = train.iloc[:,73:74]

y4 = train.iloc[:,74:]

In [4]:
test1 = test.iloc[:,:1]

test2 = test.iloc[:,1:36]

test3 = test.iloc[:,36:71]

In [5]:
x1 = x1.interpolate(axis=0)
x2 = x2.interpolate(axis=0)
x3 = x3.interpolate(axis=0)

test1 = test1.interpolate(axis=0)
test2 = test2.interpolate(axis=0)
test3 = test3.interpolate(axis=0)

In [6]:
x1 = x1.fillna(x1.mean())
x2 = x2.fillna(x2.mean())
x3 = x3.fillna(x3.mean())

test1 = test1.fillna(test1.mean())
test2 = test2.fillna(test2.mean())
test3 = test3.fillna(test3.mean())

In [7]:
x = pd.merge(x1,x2, how='left', on='id')

In [8]:
x = pd.merge(x,x3, how='left', on='id')

In [9]:
x.shape

(10000, 71)

In [10]:
test = pd.merge(test1,test2, how='left', on='id')

In [11]:
test = pd.merge(test,test3, how='left', on='id')

In [12]:
# 전처리
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = RobustScaler()
scaler.fit(x) 
x = scaler.transform(x)
test = scaler.transform(test)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# pca.fit(x)
# x = pca.transform(x)
# x_predict = pca.transform(x_predict)

In [167]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(x) 
# x = scaler.transform(x)
# x_predict = scaler.transform(x_predict)

In [14]:
# 두번째꺼 0.71
x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

thresholds = np.sort(model.feature_importances_)

for thresh in thresholds : # 컬럼수만큼 돈다! 빙글빙글
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selection_x_train = selection.transform(x_train)
    # print(selection_x_train.shape) # 칼럼이 하나씩 줄고 있는걸 알 수 있음 (가장 중요 x를 하나씩 지우고 있음)
    
    selection_model = XGBRegressor()
    selection_model.fit(selection_x_train, y_train)

    select_x_test = selection.transform(x_test)
    x_pred = selection_model.predict(select_x_test)

    mae = mean_absolute_error(y_test, x_pred)
    print('mae는',mae)

    modelpath = './model/xgb_save/auc' + str(mae) + '.dat'

C:\Users\bitcamp\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


랜덤포레스트 mae 는 0.7082098
mae는 0.7349076338338851
mae는 0.7501098101425171
mae는 0.7350139020013808
mae는 0.7308949107527732
mae는 0.741435033583641
mae는 0.7397070063066483
mae는 0.7306379121851921
mae는 0.736061611981392
mae는 0.7321029380917549
mae는 0.7333246780228615
mae는 0.7379412888479233
mae는 0.7338554907870294
mae는 0.7391021495962143
mae는 0.736312471203804
mae는 0.7382736615347862
mae는 0.7358255023241043
mae는 0.7426149238824844
mae는 0.7380071192646026
mae는 0.7390459563469888
mae는 0.7333999435138703
mae는 0.7443771235227585
mae는 0.7237133423662185
mae는 0.7412446620965004
mae는 0.7329548334932326
mae는 0.7356858549809456
mae는 0.7341450897717475
mae는 0.7318013655757905
mae는 0.7370261344099045
mae는 0.7329287794709205
mae는 0.7406329316210747
mae는 0.7378262117576598
mae는 0.7271239749360083
mae는 0.7184309677505493
mae는 0.7401643600440024
mae는 0.7272806949710846
mae는 0.7537565509343147
mae는 0.7479528230524063
mae는 0.7704446580839158
mae는 0.7655433742046356
mae는 0.7648859864664077
mae는 0.7639058007335

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

for thresh in thresholds : # 컬럼수만큼 돈다! 빙글빙글
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selection_x_train = selection.transform(x_train)
    # print(selection_x_train.shape) # 칼럼이 하나씩 줄고 있는걸 알 수 있음 (가장 중요 x를 하나씩 지우고 있음)
    
    selection_model = XGBRegressor()
    selection_model.fit(selection_x_train, y_train)

    select_x_test = selection.transform(x_test)
    x_pred = selection_model.predict(select_x_test)

    mae = mean_absolute_error(y_test, x_pred)
    print('mae는',mae)

#     modelpath = './model/xgb_save/auc' + str(mae) + '.dat'

C:\Users\bitcamp\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


랜덤포레스트 mae 는 1.2873899666666668
mae는 1.285354427176714
mae는 1.285354427176714
mae는 1.3072834283709527
mae는 1.297105118353963
mae는 1.297105118353963
mae는 1.2937072267216445
mae는 1.2937072267216445
mae는 1.2937072267216445
mae는 1.2937072267216445
mae는 1.2933081066989898
mae는 1.2933081066989898
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3046804619956014
mae는 1.3111759944450856
mae는 1.3055504852366449
mae는 1.3053963696730138
mae는 1.3053963696730138
mae는 1.2958291610592605
mae는 1.319330306689143
mae는 1.319330306689143
mae는 1.3193303

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y3, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y4, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


랜덤포레스트 mae 는 0.7106247999999998


C:\Users\bitcamp\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


랜덤포레스트 mae 는 2.189337916666666


C:\Users\bitcamp\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


랜덤포레스트 mae 는 1.409982683333333


# GradientBoostingRegressor

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)

model = GradientBoostingRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y3, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y4, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 1.497140941230232


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 0.7330654852551607


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 2.2622474812725515


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 1.4208537674215866


In [57]:
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)

model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y3, test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y4, test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 1.5842333605510353


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 0.7352794179255832


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 2.30840271440237


C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor mae 는 1.4268806905899483


# XGBRegressor()

In [27]:
y1 = pd.DataFrame(y1).reset_index
y1

<bound method DataFrame.reset_index of         hhb
id         
0      5.59
1      0.00
2     10.64
3      5.67
4     11.97
...     ...
9995  12.68
9996   8.46
9997   9.84
9998   6.38
9999   9.35

[10000 rows x 1 columns]>

In [26]:
x1 = pd.DataFrame(x).reset_index
x1

<bound method DataFrame.reset_index of        0         1         2         3         4         5         6   \
0     0.5  1.531688  1.366064  1.348957  1.179703  1.129366  1.031533   
1    -1.0 -0.243335 -0.241556 -0.184363 -0.236173 -0.241899 -0.219267   
2     0.5 -0.243335 -0.118572 -0.165968 -0.146762 -0.123755 -0.099449   
3    -1.0  1.043054  0.928122  0.763915  0.780322  0.700080  0.645123   
4    -0.5  4.505075  3.530311  2.778283  2.212718  1.815929  1.476635   
...   ...       ...       ...       ...       ...       ...       ...   
9995 -0.5  0.875889  0.890131  0.978402  1.026985  1.053623  1.063049   
9996  0.0 -0.122521 -0.206183 -0.128323 -0.201994 -0.159140 -0.178613   
9997 -1.0  2.450257  2.113281  1.913165  1.677090  1.484911  1.326271   
9998 -0.5  4.503017  3.553944  2.784231  2.209573  1.794577  1.473266   
9999 -1.0  0.900351  0.812242  0.873583  0.807319  0.775257  0.792305   

            7         8         9   ...          61         62           63  \
0    

In [29]:
pd.merge(x1,y1, how='left', on='id')

TypeError: Can only merge Series or DataFrame objects, a <class 'method'> was passed

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)

model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y3, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y4, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.2829265556406977
xgboost mae 는 0.7349076338338851
xgboost mae 는 2.2790482931423184
xgboost mae 는 1.4711147229343653


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)

model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y2, test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y3, test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y4, test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

In [88]:
model2 = joblib.load('./model_save/mae78.17250055599212.dat')

# 제출

In [42]:
# 제출
pred = model1.predict(x_predict)

pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

submission.to_csv('submission0611_1.csv')

In [255]:
submission

,hhb,hbo2,ca,na
id,,,,
10000,7.330,3.861,7.608,2.890
10001,6.530,3.993,6.310,2.105
10002,11.439,4.057,9.335,3.524
10003,8.239,4.599,9.715,3.084
10004,8.279,3.615,9.837,4.130
...,...,...,...,...
19995,6.134,4.480,9.480,3.342
19996,7.258,4.027,9.550,3.275
19997,9.724,3.333,9.786,3.206
